In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.colors as mcolors

from experiment_hyperparameters import RESULT_PATH, REPLICATION_PATH, SPLITS, SIZES

In [2]:
def group_arquitecture(name:str):
    if name.startswith("resnet"):
        return "resnet"
    elif name.startswith("vgg"):
        return "vgg"
    elif name.startswith("efficientnet"):
        return "efficientnet"
    elif name.startswith("mobilenet"):
        return "mobilenet"
    elif name.startswith("inception"):
        return "inception"
    else:
        return "proposed"

In [3]:

for split in SPLITS:
    df = pd.read_csv(os.path.join(REPLICATION_PATH, split, "consumption.csv"))
    df = df[["model", "J_per_img", "num_parameters"]]
    df["arquitecture"] = df["model"].apply(lambda x: group_arquitecture(x))

    arquitecture_codes, arquitecture_labels = pd.factorize(df['arquitecture'])

    norm = mcolors.Normalize(vmin=arquitecture_codes.min(),
                            vmax=arquitecture_codes.max())
    cmap = plt.cm.viridis
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(
        x=df['num_parameters'],
        y=df['J_per_img'],
        c=arquitecture_codes,
        cmap=cmap,
        norm=norm,
        s=100,
        alpha=0.7
    )
    for i in range(len(df)):
        plt.annotate(
            df['model'].iloc[i],
            (df['num_parameters'].iloc[i], df['J_per_img'].iloc[i]),
            textcoords="offset points",
            xytext=(5, 5),
            ha='left'
        )
    plt.xlabel('Numbers of parameters')
    plt.ylabel('Jules per image')
    plt.title(f'Jules per image vs Numbers of parameters ({split})')
    plt.grid(True)
    handles = []
    for code, arch in enumerate(arquitecture_labels):
        color = cmap(norm(code))  # same mapping as above
        handles.append(
            plt.scatter([], [], c=[color], s=100, alpha=0.7, label=arch)
        )

    plt.legend(handles=handles, title='arquitecture')
    plt.savefig(os.path.join(REPLICATION_PATH, split, f"{split}.png"))
    plt.close()

In [ ]:
kwh_per_dataset = {"size": SIZES}
for split in SPLITS:
    df = pd.read_csv(os.path.join(RESULT_PATH, split, "performance_macro.csv")).drop(columns=["Unnamed: 0"]).sort_values(by="size", ascending=True)
    kwh_per_dataset[split] = df["kwh"]
    df = df.drop(columns=["kwh"])
    plt.figure(figsize=(10, 6))
    for column in df.columns:
        if column != "size":
            plt.plot(df["size"], df[column], marker='o', label=column)

    plt.xlabel("Size")
    plt.ylabel("Metric Value")
    plt.title(f"Metrics vs. Size {split}")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(RESULT_PATH, split, f"{split}_metrics.png"))
    plt.close()

df = pd.DataFrame(kwh_per_dataset)
for column in df.columns:
    if column != "size":
        plt.plot(df["size"], df[column], marker='o', label=column)

plt.xlabel("Size")
plt.ylabel("Metric Value")
plt.title(f"Metrics vs. Size {split}")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(RESULT_PATH, f"{split}_kwh.png"))
plt.close()